In [1]:
import random
from dataclasses import replace

from replenishment import ArticleSimulationConfig, ReorderPointPolicy, simulate_replenishment_for_articles

{'article-1001': SimulationSummary(total_demand=97, total_fulfilled=87, total_backorders=10, fill_rate=0.8969072164948454, average_on_hand=16.333333333333332, holding_cost=78.4, stockout_cost=35.0, ordering_cost=19.0, total_cost=132.4), 'article-1002': SimulationSummary(total_demand=72, total_fulfilled=72, total_backorders=0, fill_rate=1.0, average_on_hand=28.5, holding_cost=136.8, stockout_cost=0.0, ordering_cost=19.0, total_cost=155.8)}


### Policy cadence parameters (review cycle + forecast horizon)

Set the order cadence (**review_period**) and the coverage window (**forecast_horizon**, excluding lead time).
The policy uses a total horizon of **lead_time + forecast_horizon**.


In [ ]:
review_period = 1
forecast_horizon = 1
rmse_window = review_period  # optional; defaults to review_period


## Example: optimize service level and review window


In [ ]:
from replenishment import (
    build_point_forecast_article_configs_from_standard_rows,
    build_replenishment_decisions_from_simulations,
    compute_backtest_rmse_by_article,
    generate_standard_simulation_rows,
    optimize_aggregation_and_service_level_factors,
    optimize_service_level_factors,
    replenishment_decision_rows_to_dataframe,
    simulate_replenishment_for_articles,
    simulate_replenishment_with_aggregation,
    split_standard_simulation_rows,
    standard_simulation_rows_to_dataframe,
)

rows = generate_standard_simulation_rows(
    n_unique_ids=3,
    periods=60,
    forecast_start_period=30,
    history_mean=20.0,
    history_std=5.0,
    forecast_mean=18.0,
    forecast_std=4.0,
    lead_time=3,
    initial_on_hand=25,
    current_stock=25,
    holding_cost_per_unit=1,
    stockout_cost_per_unit=5,
    order_cost_per_order=10,
)

backtest_rows, evaluation_rows = split_standard_simulation_rows(rows)

# Add simulated actuals for the forecast/evaluation period.
rng = random.Random(7)
def _sample_int(mean: float, std: float) -> int:
    return max(0, int(round(rng.gauss(mean, std))))

evaluation_rows = [
    replace(row, actuals=_sample_int(20.0, 5.0), demand=_sample_int(20.0, 5.0))
    for row in evaluation_rows
]

rows_df = standard_simulation_rows_to_dataframe(
    backtest_rows + evaluation_rows, library="pandas"
)


In [ ]:
candidate_service_levels = [0.8, 0.9, 0.95]
backtest_rmse = compute_backtest_rmse_by_article(
    backtest_rows,
    rmse_window=rmse_window,
)

point_configs = build_point_forecast_article_configs_from_standard_rows(
    backtest_rows,
    service_level_factor=candidate_service_levels[0],
    service_level_mode="service_level",
    fixed_rmse=backtest_rmse,
    review_period=review_period,
    forecast_horizon=forecast_horizon,
    rmse_window=rmse_window,
)

optimized = optimize_service_level_factors(
    point_configs,
    candidate_factors=candidate_service_levels,
    service_level_mode="service_level",
)
best_factors = {
    unique_id: result.service_level_factor
    for unique_id, result in optimized.items()
}

backtest_actuals = {}
for row in backtest_rows:
    backtest_actuals.setdefault(row.unique_id, []).append(row)
for unique_id, series in backtest_actuals.items():
    series.sort(key=lambda r: r.ds)
    backtest_actuals[unique_id] = [int(r.actuals) for r in series]

eval_configs = build_point_forecast_article_configs_from_standard_rows(
    evaluation_rows,
    service_level_factor=best_factors,
    service_level_mode="service_level",
    fixed_rmse=backtest_rmse,
    actuals_override=backtest_actuals,
    review_period=review_period,
    forecast_horizon=forecast_horizon,
    rmse_window=rmse_window,
)
eval_simulations = simulate_replenishment_for_articles(eval_configs)
eval_decisions = build_replenishment_decisions_from_simulations(
    evaluation_rows,
    eval_simulations,
    review_period=review_period,
    forecast_horizon=forecast_horizon,
    rmse_window=rmse_window,
    sigma=best_factors,
    fixed_rmse=backtest_rmse,
    service_level_mode="service_level",
)

decisions_df = replenishment_decision_rows_to_dataframe(eval_decisions)
decisions_df.head()


In [ ]:
candidate_windows = [1, 2, 3]
agg_sigma_result = optimize_aggregation_and_service_level_factors(
    point_configs,
    candidate_windows=candidate_windows,
    candidate_factors=[0.9],
    service_level_mode="service_level",
)
agg_windows = {
    unique_id: result.window
    for unique_id, result in agg_sigma_result.items()
}

agg_simulations = {
    unique_id: simulate_replenishment_with_aggregation(
        periods=config.periods,
        demand=config.demand,
        initial_on_hand=config.initial_on_hand,
        lead_time=config.lead_time,
        policy=config.policy,
        aggregation_window=agg_windows[unique_id],
        holding_cost_per_unit=config.holding_cost_per_unit,
        stockout_cost_per_unit=config.stockout_cost_per_unit,
        order_cost_per_order=config.order_cost_per_order,
        order_cost_per_unit=config.order_cost_per_unit,
    )
    for unique_id, config in eval_configs.items()
}

agg_decisions = build_replenishment_decisions_from_simulations(
    evaluation_rows,
    agg_simulations,
    aggregation_window=agg_windows,
    review_period=agg_windows,
    forecast_horizon=agg_windows,
    rmse_window=agg_windows,
    sigma=best_factors,
    fixed_rmse=backtest_rmse,
    service_level_mode="service_level",
)
agg_decisions_df = replenishment_decision_rows_to_dataframe(agg_decisions)
agg_decisions_df.head()
